# 🛠️ Utilização Avançada de Ferramentas com Modelos GitHub (.NET)

## 📋 Objetivos de Aprendizagem

Este notebook demonstra padrões de integração de ferramentas de nível empresarial utilizando o Microsoft Agent Framework em .NET com Modelos GitHub. Vais aprender a construir agentes sofisticados com múltiplas ferramentas especializadas, aproveitando a tipagem forte do C# e as funcionalidades empresariais do .NET.

**Capacidades Avançadas de Ferramentas que Vais Dominar:**
- 🔧 **Arquitetura Multi-Ferramenta**: Construção de agentes com múltiplas capacidades especializadas
- 🎯 **Execução de Ferramentas com Tipagem Segura**: Aproveitamento da validação em tempo de compilação do C#
- 📊 **Padrões de Ferramentas Empresariais**: Design de ferramentas pronto para produção e gestão de erros
- 🔗 **Composição de Ferramentas**: Combinação de ferramentas para fluxos de trabalho empresariais complexos

## 🎯 Benefícios da Arquitetura de Ferramentas em .NET

### Funcionalidades Empresariais de Ferramentas
- **Validação em Tempo de Compilação**: Tipagem forte garante a correção dos parâmetros das ferramentas
- **Injeção de Dependências**: Integração com contêiner IoC para gestão de ferramentas
- **Padrões Async/Await**: Execução de ferramentas não bloqueante com gestão adequada de recursos
- **Registo Estruturado**: Integração de registo incorporado para monitorização da execução de ferramentas

### Padrões Prontos para Produção
- **Gestão de Exceções**: Gestão abrangente de erros com exceções tipadas
- **Gestão de Recursos**: Padrões adequados de eliminação e gestão de memória
- **Monitorização de Desempenho**: Métricas incorporadas e contadores de desempenho
- **Gestão de Configuração**: Configuração com tipagem segura e validação

## 🔧 Arquitetura Técnica

### Componentes Centrais de Ferramentas em .NET
- **Microsoft.Extensions.AI**: Camada de abstração unificada para ferramentas
- **Microsoft.Agents.AI**: Orquestração de ferramentas de nível empresarial
- **Integração com Modelos GitHub**: Cliente API de alto desempenho com pooling de conexões

### Pipeline de Execução de Ferramentas
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Categorias de Ferramentas & Padrões

### 1. **Ferramentas de Processamento de Dados**
- **Validação de Entrada**: Tipagem forte com anotações de dados
- **Operações de Transformação**: Conversão e formatação de dados com tipagem segura
- **Lógica Empresarial**: Ferramentas de cálculo e análise específicas do domínio
- **Formatação de Saída**: Geração de respostas estruturadas

### 2. **Ferramentas de Integração** 
- **Conectores API**: Integração com serviços RESTful utilizando HttpClient
- **Ferramentas de Base de Dados**: Integração com Entity Framework para acesso a dados
- **Operações de Ficheiros**: Operações seguras no sistema de ficheiros com validação
- **Serviços Externos**: Padrões de integração com serviços de terceiros

### 3. **Ferramentas Utilitárias**
- **Processamento de Texto**: Utilitários de manipulação e formatação de strings
- **Operações de Data/Hora**: Cálculos de data/hora sensíveis à cultura
- **Ferramentas Matemáticas**: Cálculos de precisão e operações estatísticas
- **Ferramentas de Validação**: Validação de regras empresariais e verificação de dados

## ⚙️ Pré-requisitos & Configuração

**Ambiente de Desenvolvimento:**
- SDK .NET 9.0 ou superior
- Visual Studio 2022 ou VS Code com extensão C#
- Acesso à API de Modelos GitHub

**Pacotes NuGet Necessários:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Configuração de Ambiente (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Pronto para construir agentes de nível empresarial com capacidades poderosas e ferramentas com tipagem segura em .NET? Vamos arquitetar soluções profissionais! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se uma tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
